### Imports

In [1]:
from stimulus_helpers import *
import pandas as pd
import os

### Function to make mistmatched stimuli

In experiment 3, we showed valid attention sequences over the wrong image backgrounds ("mismatched" background images). Here we define functions to create mismatched stimuli.

In [3]:
def movie_maker_mismatch(fixations, dictionary, this_list, this_version, stim_type, isolated=False, background=[], background_list=[]):
    '''
    INPUTS:
    
    fixations  - pandas dataframe containing only fixation data from one subject viewing one image
    dictionary - dictionary containing two keys, one with the image name and one with the fixaiton coordinates 
    stim_type  - string describing the type of attention stimulus to make: veridical / scrambled / mismatched / reversed
    isolated   - boolean describing whether the attention will be isolated (True) or be overlaid on an image (False)
            
    OUTPUTS: 
    
    technically, none
    
    This will create and save two files: 
    - mp4 video of the attention video stimulus
    - csv containing video metadata
        - # of attention chunks (num_chunks)
        - order of attention chunks (order)
        - image filename (dictionary['image'])
        - subject ID (dictionary['subject'])
    '''
    
    # This function can make veridical and scrambled attention spotlight videos, isolated or overlaid on an image

    lengths = []; movie_frames = []; total_counter = 0
    order   = list(set(fixations))
    num_chunks = len(order)

    if len(order)>2:

        # scramble order for scrambled stimulus
        if stim_type=='scrambled':
            while order == list(range(1, len(order)+1)):
                remainder = order[1:]
                np.random.shuffle(remainder)
                new_list = [order[0]] + list(remainder)
                order = new_list

        # record how many chunks stay in the same place (besides the first one), if any
        number_same = sum(x == y for x, y in zip(order, list(range(1, len(order)+1))))-1
            
        # for each fixation chunk 
        for x in order:
    
            length_counter = 0; tuple_list = []; total_list = []

            # for first chunk, collect the largest and smallest x and y vals
            if x==1:
            	x1_vals=[100000000000,0]; y1_vals=[100000000000,0]
            	# initialize very high and very low values, respectively

            # for each item in fixie chunk list AND each tuple
            for a,b in zip(fixations, dictionary['fixations']):

                # if item from fixie chunk list is the number from first for statement:
                if a == x:
                    tuple_list.append(b)
                    total_counter += 1
                    total_list.append(total_counter)
                    if x==1:
                    	x1_vals,y1_vals = first_bounds_update(x1_vals,y1_vals,b)
            
            # determine if first hotspot contains center of screen
            first_centered   = one_centered(x1_vals, y1_vals)
            reverse_centered = one_centered(y1_vals, x1_vals)

            # pass into plotting function
            plot_heatmap_mismatch({'image':dictionary['image'], 'fixations':tuple_list}, 
                         filename='/Users/kirstenziman/Downloads/Images_resized_greyborders/List'+str(this_list)+'/' + dictionary['image'], 
                         alpha=.6, cmap="Greys_r", clean=False, isolated=isolated, other=True, l=this_list,
                                 background=background, background_list=background_list)
            # images_with_borders/
            
            # save jpegs    
            for q in total_list:
                addin = get_addin(q)
                plt.savefig(dictionary['image']+addin+str(q)+'.jpg')

        # determine frame rate given number of images
        framerate = get_framerate()

        if framerate!=0:
            ending = '_'+dictionary['subject'][0]+'_L'+str(this_list)+'_V'+str(this_version)+'_'+stim_type+'_freeview_iso'+str(isolated)+'mismatch_'+background+'_L'+str(background_list)

        # compile and save video
            (
            ffmpeg
            .input('*.jpg', pattern_type='glob', framerate=framerate)
            .output(dictionary['image']+ending+'.mp4')
            .run()
            )
            
            # remove leftover jpeg images
            remove_jpegs()

        else:
            print('framerate is zero for '+dictionary['image'])

            for file_name in os.listdir('.'):
                if file_name.endswith('.jpg'):
                    os.remove(file_name)

        meta = pd.DataFrame({'image':dictionary['image'],'subject':dictionary['subject'],'num_chunks':num_chunks, 'order':[order], 'center_first':str(first_centered), 'center_first_reverse':str(reverse_centered), 'chunks_same': number_same, 'proportion_same':number_same/(len(order)-1), 'x1_vals':[x1_vals], 'y1_vals':[y1_vals],
                            'background_image':background,'background_list':background_list})

        meta.to_csv(dictionary['image']+ending+'.csv')

    else:
        print('len(k) <= 2: '+dictionary['image'])

In [4]:
# KZ added "isolated attention" option 
# def plot_heatmap(data, filename='test.pdf', alpha=0.7, cmap='jet', clean=True):
def plot_heatmap_mismatch(data, filename='test.pdf', alpha=0.7, cmap='jet', clean=True, isolated=False, other=None, l=1, background=[], background_list=[]):
    """
    Plot heatmap.
    Mostly modified from PyGaze.
    """
    # Load data
    image      = data['image']
    image_path = '/Users/kirstenziman/Downloads/Images_resized_greyborders/List '+str(l)+'/'+data['image'] # IMAGE_PATHS[image]
    #KZ update from ndimage to imageio #image_data = ndimage.imread(image_path)
    # images_with_borders/
    image_data = imageio.imread(image_path)
    heatmap    = buildFixMap(data['fixations'])
    # Remove haze.
    if clean:
        heatmap = clean_heatmap(heatmap)
    
    # Matplotlib.
    # Borrows heavily from from Edwin Dalmaijer's `gazeplotter.py` script, from the PyGaze codebase.
    dpi = 100
    display_size = heatmap.shape
    figsize = (display_size[1]/dpi, display_size[0]/dpi)
    fig = plt.figure(figsize=figsize, dpi=dpi, frameon=False)
    ax = plt.Axes(fig, [0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    if isolated == False and other == None:
        ax.imshow(image_data)
        # KZ moved ax.imshow(image_data) into if statement for "isolated attention" option
    
    elif other != None:
        new_image_path = '/Users/kirstenziman/Downloads/Images_resized_greyborders/List '+str(background_list)+'/'+background
        new_image_data = imageio.imread(new_image_path)
        ax.imshow(new_image_data)
        
    ax.imshow(heatmap, cmap=cmap, alpha=alpha)

We use the viewer-attention pairs from experiments 1 and 2

In [31]:
# get the image info from stim_list_1
list1 = [x for x in os.listdir('/Users/kirstenziman/Documents/github/predicting_attention_MRI/pilot_experiment/STIM_SET_1_144/') if x[-6:]=='44.mp4' and 'veridical' in x]

# get the image info from stim_list_2
list2 = [x for x in os.listdir('/Users/kirstenziman/Documents/predicting_attention/experiments/experiment_6/STIM_SET_2_144/') if x[-6:]=='44.mp4' and 'veridical' in x]


### Get images

In [32]:
list2_image = [x[:11] for x in list2]

for idx,x in enumerate(list2_image):
    if x[-1]=='_':
        list2_image[idx]=x[:-1]

In [33]:
list1_image = [x[:11] for x in list1]

for idx,x in enumerate(list1_image):
    if x[-1]=='_':
        list1_image[idx]=x[:-1]

In [34]:
for idx,x in enumerate(list1_image):
    if x[-2:]=='_p':
        list1_image[idx]=x[:-2]

### Get subjects

In [35]:
list2_sub = [x[11:17] for x in list2]

for idx,x in enumerate(list2_sub):
    if x[0]=='_':
        list2_sub[idx]=x[1:]
        
    if list2_sub[idx][-1]=='L':
        list2_sub[idx]=list2_sub[idx][:-1]
        
    if list2_sub[idx][-1]=='_':
        list2_sub[idx]=list2_sub[idx][:-1]
        
#list2_sub

In [36]:
list1_sub = [x[10:17] for x in list1]

for idx,x in enumerate(list1_sub):
    if x[0]=='_':
        list1_sub[idx]=x[1:]
        
    if x[0:2]=='p_':
        list1_sub[idx]=x[2:]
        
    if list1_sub[idx][-1]=='L':
        list1_sub[idx]=list1_sub[idx][:-1]
        
    if list1_sub[idx][-1]=='_':
        list1_sub[idx]=list1_sub[idx][:-1]
        
    if list1_sub[idx][-3:]=='_L3':
        list1_sub[idx]=list1_sub[idx][:-3]
        
# list1_sub

In [37]:
indices  = [x.index('L') for x in list1]

listies1 = []

for x,y in zip(indices,list1):
    listies1.append(int(y[x+1:x+2]))

In [38]:
indices  = [x.index('L') for x in list2]

listies2 = []

for x,y in zip(indices,list2):
    listies2.append(int(y[x+1:x+2]))

### Make the mismatches

Now that we have a list of the subjects, images, and list numbers, we can make the stimulus dataframe

In [42]:
len(listies2)

79

In [43]:
df = pd.DataFrame({'image':list1_image, 'subject':list1_sub, 
                  'list':listies1,'background_image':list2_image[0:70],
                 'back_sub':list2_sub[0:70], 'back_list':listies2[0:70]})

In [44]:
df

,image,subject,list,background_image,back_sub,back_list
0,498149.bmp,pp99,2,1160189.bmp,pp64,1
1,712977.bmp,pp51,1,1159434.bmp,pp62,2
2,1592362.bmp,pp95,2,1159575.bmp,pp62,2
3,1592890.bmp,pp82,2,1592305.bmp,pp65,3
4,713135.bmp,pp54,2,285932.bmp,pp64,1
...,...,...,...,...,...,...
65,1592804.bmp,pp91,3,713920.bmp,pp76,1
66,286001.bmp,pp54,2,1592921.bmp,pp69,3
67,1591961.bmp,pp67,2,1159956.bmp,pp59,3
68,498155.bmp,pp61,2,1159329.bmp,pp61,2


In [49]:
# df = pd.read_csv('stim10.csv') 
counter = 0

# lr   = list(range(0,70))
# random.shuffle(lr) 

for idx,row in df.iterrows():
    
    free_view   = get_gaze(row['subject'],int(row['list']),1,row['image'])
    free_fixies = get_fixies(free_view) 
    free_dicts  = make_the_dicts(free_view)

    movie_maker_mismatch(free_fixies, free_dicts, row['list'], 1, 'veridical', 
                         isolated=False, background=row['background_image'], 
                         background_list=row['back_list'])
                                                      
    counter +=1; print(counter)

In [45]:
# def get_gaze(sub, this_list, this_version, image):
#     '''
#     INPUTS:
#     sub          - string indicating subject ID (example: 'pp151')
#     this_list    - int indicating which List the image is from (1, 2, or 3)
#     this_version - int indicating which Version of this_list the data is from (1 or 2)
    
#     OUTPUTS:
#     gaze_data    - df containing sub's full gaze data for image 
#     '''

#     subdir     = '../Free_viewing/List'+str(this_list)+'_ALL/'+sub+'/eye'
#     eye_trials = os.listdir(subdir)
#     null       = 0

#     for trial in eye_trials:
#         if trial!='.DS_Store' and trial[-3:]!='csv':

#             dat_string = subdir+'/'+trial
#             splitso    = make_splitso(sub, dat_string, this_list, this_version)

#             if 'Stimulus' in splitso.columns:
#                 if splitso[splitso['Stimulus']==image].shape[0]>0:
#                     free_view = splitso[splitso['Stimulus']==image]
#                     null+=1
                
# #             else:
# #                 if warning_counter == 0:
# #                     print('No stimulus column in the df')
# #                     print(sub)
# #                     print(this_list)
# #                     print(this_version)
# #                     #print(splitso.head())

#     if null==0:
#         print("Reportedly, "+sub+" has no values in df for this stimulus: "+image)
#         print(sub)
#         print(this_list)
#         print(this_version)
#         free_view = []
#         print(splitso.head())
        
#     return(free_view)